In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
import string
import re

In [2]:
## import data file

physics = pd.read_csv('Datasets/test.csv')

In [3]:
physics.head()

,id,title,content
0,1,What is spin as it relates to subatomic partic...,<p>I often hear about subatomic particles havi...
1,2,What is your simplest explanation of the strin...,<p>How would you explain string theory to non ...
2,3,"Lie theory, Representations and particle physics",<p>This is a question that has been posted at ...
3,7,Will Determinism be ever possible?,<p>What are the main problems that we need to ...
4,9,Hamilton's Principle,<p>Hamilton's principle states that a dynamic ...


In [10]:
physics.shape

(81926, 6)

In [5]:
## concatenates title and content with a space in the middle
physics['all_text'] = physics['title'] + " " + physics['content']
## makes 'all_text' lowercase
physics['all_text'] = physics['all_text'].apply(lambda x: x.lower())


In [6]:
## removes punctuation from a string
def remove_punctuation(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

In [7]:
## removes html garbage from a string

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = cleantext.replace('\n', '')
    return cleantext

In [9]:
## removes html tags from all_text
physics['no_html'] = physics['all_text'].apply(cleanhtml)
## removes punctuation from all_text
physics['no_punctuation'] = physics['no_html'].apply(remove_punctuation)

In [18]:
## instantiates a tfidf object
tvec = TfidfVectorizer(stop_words='english')
## fits the cleaned text to tfidf
tvec.fit(physics['no_punctuation'])

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [19]:
## turns the fit tfidf data into a data frame
test = pd.DataFrame(tvec.transform(physics['no_punctuation']).todense(),
                   columns=tvec.get_feature_names())

In [20]:
def get_best_word(df, index):
    row = df.iloc[index, :]
    row_tuple = zip(row.index, row.values)
    return max(row_tuple, key=lambda x: x[1])

In [ ]:
## runs get_best_word on each row of data and returns a list of tuples
## started 16:19

list_of_tags = []

for i in range(len(test)):
    list_of_tags.append(get_best_word(test, i))
    print i